In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
from tqdm import tqdm

/Users/aaron/Documents/ml/kaggle/venv/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [6]:
N_CLASSES = 10
LR = 0.001
EPOCHS = 1
BATCH_SIZE = 32

In [2]:
def load_data():
    mnist = input_data.read_data_sets("MNIST_data/")
    train_x = mnist.train.images
    train_y = np.array(mnist.train.labels,  dtype=np.int32)
    test_x = mnist.test.images
    test_y = np.array(mnist.test.labels, dtype=np.int32)
    return (train_x, train_y), (test_x, test_y)

(train_x, train_y), (test_x, test_y) = load_data()

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [3]:
type(train_x)

numpy.ndarray

In [4]:
train_x.shape

(55000, 784)

In [7]:
def get_train_ds(features, labels):
    dataset = tf.data.Dataset.from_tensor_slices((features, labels))
    return dataset.shuffle(1000).repeat().batch(BATCH_SIZE)

train_ds = get_train_ds(train_x, train_y)
train_ds

<BatchDataset shapes: ((?, 784), (?,)), types: (tf.float32, tf.int32)>

In [8]:
iterator = train_ds.make_one_shot_iterator()

In [9]:
next_item = iterator.get_next()
next_item

(<tf.Tensor 'IteratorGetNext:0' shape=(?, 784) dtype=float32>,
 <tf.Tensor 'IteratorGetNext:1' shape=(?,) dtype=int32>)

In [10]:
if 'sess' not in locals():
    sess = tf.InteractiveSession()
sess

In [11]:
x_train_batch, y_train_batch = sess.run(next_item)

In [22]:
print(type(x_train_batch))
print(x_train_batch.shape)
print(type(y_train_batch))
print(y_train_batch.shape)

<class 'numpy.ndarray'>
(32, 784)
<class 'numpy.ndarray'>
(32,)


In [16]:
X = tf.placeholder(tf.float32, shape=[None, 784], name="X")
y = tf.placeholder(tf.float32, shape=[None], name="y")

In [46]:
def conv_net(features):
    net = tf.layers.dense(features, 100, activation='relu')
    return tf.layers.dense(net, N_CLASSES)

def get_train_op(logits, labels):
    loss_op = tf.reduce_mean(
        tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits=logits, labels=labels))
    optimizer = tf.train.AdamOptimizer(learning_rate=LR)
    train_op = optimizer.minimize(loss_op, global_step=tf.train.get_global_step())
    return loss_op, train_op

def get_test_ds(features, labels):
    dataset = tf.data.Dataset.from_tensor_slices((features, labels))
    return dataset.batch(BATCH_SIZE)

In [59]:
logits_train = conv_net(X)
logits_test = conv_net(X)

# predictions
pred_classes = tf.argmax(logits_test, axis=1)
# pred_probas = tf.nn.softmax(pred_classes)

# train_op
loss_op = tf.reduce_mean(
    tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits_train, labels=pred_classes))
optimizer = tf.train.AdamOptimizer(learning_rate=LR)
train_op = optimizer.minimize(loss_op)

# accuracy_op - for test set
accuracy_op = tf.metrics.accuracy(labels=y, predictions=pred_classes)

# init global + local VARs
sess.run(tf.global_variables_initializer())
sess.run(tf.local_variables_initializer())

In [50]:
EPOCHS = 3
# for i in tqdm(range(EPOCHS)):

train_ds = get_train_ds(train_x, train_y)
train_next = train_ds.make_one_shot_iterator().get_next()

test_ds = get_test_ds(test_x, test_y)
test_next = test_ds.make_one_shot_iterator().get_next()

for step in tqdm(range(train_x.shape[0] // BATCH_SIZE)):
    x_train_batch, y_train_batch = sess.run(train_next)
    loss, train = sess.run(
        [loss_op, train_op],
        feed_dict={X: x_train_batch, y: y_train_batch})

for step in tqdm(range(test_x.shape[0] // BATCH_SIZE)):
    x_test_batch, y_test_batch = sess.run(test_next)
    test_loss, test_acc = sess.run(
        [loss_op, accuracy_op],
        feed_dict={X: x_test_batch, y: y_test_batch})




  0%|          | 0/1718 [00:00<?, ?it/s]


  0%|          | 1/1718 [00:22<10:35:36, 22.21s/it]


  2%|▏         | 31/1718 [00:22<20:13,  1.39it/s]  


  3%|▎         | 55/1718 [00:22<11:17,  2.45it/s]


  5%|▌         | 86/1718 [00:22<07:07,  3.82it/s]


  7%|▋         | 119/1718 [00:22<05:03,  5.26it/s]


  9%|▊         | 149/1718 [00:22<03:59,  6.56it/s]


 11%|█         | 184/1718 [00:22<03:10,  8.07it/s]


 13%|█▎        | 220/1718 [00:22<02:36,  9.60it/s]


 15%|█▍        | 251/1718 [00:23<02:14, 10.90it/s]


 16%|█▋        | 282/1718 [00:23<01:58, 12.15it/s]


 18%|█▊        | 308/1718 [00:23<01:46, 13.20it/s]


 19%|█▉        | 333/1718 [00:23<01:37, 14.20it/s]


 21%|██        | 364/1718 [00:23<01:27, 15.46it/s]


 23%|██▎       | 397/1718 [00:23<01:18, 16.79it/s]


 25%|██▍       | 425/1718 [00:23<01:12, 17.90it/s]


 26%|██▋       | 453/1718 [00:23<01:06, 18.99it/s]


 28%|██▊       | 482/1718 [00:23<01:01, 20.12it/s]


 30%|██▉       | 510/1718 [00:24<00:57, 21.18it/s]




In [51]:
loss

0.0015245578

In [52]:
test_loss

0.0057472405

In [53]:
accuracy

(0.11038853, 0.110392265)

In [54]:
writer = tf.summary.FileWriter('.')
writer.add_graph(tf.get_default_graph())